<a href="https://colab.research.google.com/github/pbanavara/transformers/blob/main/nano_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [ ]:
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [ ]:
print("length of characters ", len(text))

In [ ]:
print(text[:100])

In [ ]:
chars = sorted(list(set(text)))
len(chars)

In [ ]:
stoi = {ch : i for i,ch in enumerate(chars)}
itos = {i : ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[x] for x in s] # Takes a string, outputs a list of integers
decode = lambda l: ''.join([itos[i] for i in l])

In [ ]:
print(encode("hi there"))
print(decode(encode("hi there")))

In [ ]:
import torch

In [ ]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

In [ ]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
train_data[:block_size+1]

In [ ]:
x = train_data[:block_size+1]
y = train_data[1:block_size+1:]
x, y

In [ ]:
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print("when input is {} target is {}".format(context, target))

In [ ]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x,y

xb, yb = get_batch('train')
print("inputs shape {}".format(xb.shape))
print("targets shape {}".format(yb.shape))


In [ ]:
#The bigram language model
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets):
    logits = self.token_embedding_table(idx)
    B, T, C = logits.shape
    logits = logits.view(B*T, C)
    targets = targets.view(B*T)
    loss = F.cross_entropy(logits, targets)
    return logits, loss

  def generate(self, idx, max_new_tokens):
    #idx is the (B, T) array of indices in the current context


vocab_size = len(chars)
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(loss)